In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [6]:
pd.set_option('display.max_columns',None)

In [7]:
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])
#user_app_usage = pd.read_csv("user_app_usage.csv")
app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])

In [8]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    cats = list(set(app_info['category']))
    category2id = dict(zip(sorted(cats), range(0,len(cats))))
    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid')    

            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']


            df = df.merge(app_info, how='left', on='appId')
            now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame(), how='left', on='uid')
            #print(df.groupby(['uid'])['category'].value_counts().index[0])
            now_df['usage_most_used_category'] = df.groupby(['uid'])['category'].transform(f)
            resTable = pd.concat([resTable, now_df])
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [9]:
processUserAppUsage()

index: 1


C:\Users\13224\AppData\Roaming\Python\Python27\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 101
index:

In [10]:
#将user_basic_info.csv 和 user_behavior_info.csv中的字符值编码成可以训练的数值类型，合并
class2id = {}
id2class = {}
def mergeBasicTables(baseTable):
    resTable = baseTable.merge(user_basic_info, how='left', on='uid', suffixes=('_base0', '_ubaf'))
    resTable = resTable.merge(user_behavior_info, how='left', on='uid', suffixes=('_base1', '_ubef'))
    cat_columns = ['city','prodName','color','carrier','os','ct']
    for c in cat_columns:
        resTable[c] = resTable[c].apply(lambda x: x if type(x)==str else str(x))
        sort_temp = sorted(list(set(resTable[c])))  
        class2id[c+'2id'] = dict(zip(sort_temp, range(1, len(sort_temp)+1)))
        id2class['id2'+c] = dict(zip(range(1,len(sort_temp)+1), sort_temp))
        resTable[c] = resTable[c].apply(lambda x: class2id[c+'2id'][x])
        
    return resTable

In [11]:
#处理app使用相关数据
#对user_app_actived.csv简单统计
#将之前训练的appuseProcess.csv进行合并
def mergeAppData(baseTable):
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    resTable['appId'] = resTable['appId'].apply(lambda x: len(list(x.split('#'))))
    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    resTable = resTable.fillna(0)
    #print(resTable[:5])
    return resTable

In [12]:
#合并用户基本特征以及app使用相关特征，作为训练集和测试集
df_train = mergeAppData(mergeBasicTables(age_train))
df_test = mergeAppData(mergeBasicTables(age_test))
print(df_train.shape)
print(df_test.shape)

(2010000, 34)
(502500, 33)


In [13]:
#训练模型

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [19]:
print("训练模型：")
param = {
        'learning_rate': 0.11,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.9,
        'metric': 'multi_error'
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=1500, valid_sets=[vali], early_stopping_rounds=100)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission.csv',index=False)

训练模型：
0
training start...
[1]	valid_0's multi_error: 0.62613
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_error: 0.61922
[3]	valid_0's multi_error: 0.617748
[4]	valid_0's multi_error: 0.614696
[5]	valid_0's multi_error: 0.614353
[6]	valid_0's multi_error: 0.612177
[7]	valid_0's multi_error: 0.611383
[8]	valid_0's multi_error: 0.611095
[9]	valid_0's multi_error: 0.610063
[10]	valid_0's multi_error: 0.608651
[11]	valid_0's multi_error: 0.608241
[12]	valid_0's multi_error: 0.60739
[13]	valid_0's multi_error: 0.606692
[14]	valid_0's multi_error: 0.606063
[15]	valid_0's multi_error: 0.605124
[16]	valid_0's multi_error: 0.604598
[17]	valid_0's multi_error: 0.603936
[18]	valid_0's multi_error: 0.603359
[19]	valid_0's multi_error: 0.60235
[20]	valid_0's multi_error: 0.601454
[21]	valid_0's multi_error: 0.600939
[22]	valid_0's multi_error: 0.60041
[23]	valid_0's multi_error: 0.599671
[24]	valid_0's multi_error: 0.599138
[25]	valid_0's multi_error: 0.598374


[218]	valid_0's multi_error: 0.561411
[219]	valid_0's multi_error: 0.56136
[220]	valid_0's multi_error: 0.561359
[221]	valid_0's multi_error: 0.561295
[222]	valid_0's multi_error: 0.56128
[223]	valid_0's multi_error: 0.561251
[224]	valid_0's multi_error: 0.561172
[225]	valid_0's multi_error: 0.561114
[226]	valid_0's multi_error: 0.56109
[227]	valid_0's multi_error: 0.561026
[228]	valid_0's multi_error: 0.560928
[229]	valid_0's multi_error: 0.560838
[230]	valid_0's multi_error: 0.560831
[231]	valid_0's multi_error: 0.560732
[232]	valid_0's multi_error: 0.560687
[233]	valid_0's multi_error: 0.560696
[234]	valid_0's multi_error: 0.560572
[235]	valid_0's multi_error: 0.560593
[236]	valid_0's multi_error: 0.560601
[237]	valid_0's multi_error: 0.56049
[238]	valid_0's multi_error: 0.560425
[239]	valid_0's multi_error: 0.56036
[240]	valid_0's multi_error: 0.560325
[241]	valid_0's multi_error: 0.56032
[242]	valid_0's multi_error: 0.560207
[243]	valid_0's multi_error: 0.56015
[244]	valid_0's mul

[435]	valid_0's multi_error: 0.555159
[436]	valid_0's multi_error: 0.555063
[437]	valid_0's multi_error: 0.555056
[438]	valid_0's multi_error: 0.555016
[439]	valid_0's multi_error: 0.555026
[440]	valid_0's multi_error: 0.555007
[441]	valid_0's multi_error: 0.554981
[442]	valid_0's multi_error: 0.55498
[443]	valid_0's multi_error: 0.554972
[444]	valid_0's multi_error: 0.554996
[445]	valid_0's multi_error: 0.555007
[446]	valid_0's multi_error: 0.555016
[447]	valid_0's multi_error: 0.555022
[448]	valid_0's multi_error: 0.555038
[449]	valid_0's multi_error: 0.554992
[450]	valid_0's multi_error: 0.554948
[451]	valid_0's multi_error: 0.55492
[452]	valid_0's multi_error: 0.554938
[453]	valid_0's multi_error: 0.554934
[454]	valid_0's multi_error: 0.554929
[455]	valid_0's multi_error: 0.554945
[456]	valid_0's multi_error: 0.554878
[457]	valid_0's multi_error: 0.554808
[458]	valid_0's multi_error: 0.554808
[459]	valid_0's multi_error: 0.554807
[460]	valid_0's multi_error: 0.554799
[461]	valid_0'

[651]	valid_0's multi_error: 0.553392
[652]	valid_0's multi_error: 0.553356
[653]	valid_0's multi_error: 0.553377
[654]	valid_0's multi_error: 0.553313
[655]	valid_0's multi_error: 0.553319
[656]	valid_0's multi_error: 0.553341
[657]	valid_0's multi_error: 0.553359
[658]	valid_0's multi_error: 0.553377
[659]	valid_0's multi_error: 0.55338
[660]	valid_0's multi_error: 0.553374
[661]	valid_0's multi_error: 0.55335
[662]	valid_0's multi_error: 0.553339
[663]	valid_0's multi_error: 0.553326
[664]	valid_0's multi_error: 0.553299
[665]	valid_0's multi_error: 0.553351
[666]	valid_0's multi_error: 0.553316
[667]	valid_0's multi_error: 0.55332
[668]	valid_0's multi_error: 0.553281
[669]	valid_0's multi_error: 0.55326
[670]	valid_0's multi_error: 0.553253
[671]	valid_0's multi_error: 0.553287
[672]	valid_0's multi_error: 0.553248
[673]	valid_0's multi_error: 0.553241
[674]	valid_0's multi_error: 0.553226
[675]	valid_0's multi_error: 0.55321
[676]	valid_0's multi_error: 0.553204
[677]	valid_0's m

[868]	valid_0's multi_error: 0.552398
[869]	valid_0's multi_error: 0.55238
[870]	valid_0's multi_error: 0.552386
[871]	valid_0's multi_error: 0.552386
[872]	valid_0's multi_error: 0.552436
[873]	valid_0's multi_error: 0.552435
[874]	valid_0's multi_error: 0.552431
[875]	valid_0's multi_error: 0.552423
[876]	valid_0's multi_error: 0.552416
[877]	valid_0's multi_error: 0.552459
[878]	valid_0's multi_error: 0.552441
[879]	valid_0's multi_error: 0.552413
[880]	valid_0's multi_error: 0.552439
[881]	valid_0's multi_error: 0.552428
[882]	valid_0's multi_error: 0.552404
[883]	valid_0's multi_error: 0.552447
[884]	valid_0's multi_error: 0.552445
[885]	valid_0's multi_error: 0.552454
[886]	valid_0's multi_error: 0.552447
[887]	valid_0's multi_error: 0.552426
[888]	valid_0's multi_error: 0.552456
[889]	valid_0's multi_error: 0.552477
[890]	valid_0's multi_error: 0.552459
[891]	valid_0's multi_error: 0.552441
[892]	valid_0's multi_error: 0.552504
[893]	valid_0's multi_error: 0.552483
[894]	valid_0

...

[616]	valid_0's multi_error: 0.554578
[617]	valid_0's multi_error: 0.554575
[618]	valid_0's multi_error: 0.554572
[619]	valid_0's multi_error: 0.554544
[620]	valid_0's multi_error: 0.554569
[621]	valid_0's multi_error: 0.554566
[622]	valid_0's multi_error: 0.55455
[623]	valid_0's multi_error: 0.554569
[624]	valid_0's multi_error: 0.554549
[625]	valid_0's multi_error: 0.554558
[626]	valid_0's multi_error: 0.554525
[627]	valid_0's multi_error: 0.554464
[628]	valid_0's multi_error: 0.55451
[629]	valid_0's multi_error: 0.554508
[630]	valid_0's multi_error: 0.554486
[631]	valid_0's multi_error: 0.554449
[632]	valid_0's multi_error: 0.554437
[633]	valid_0's multi_error: 0.554472
[634]	valid_0's multi_error: 0.554432
[635]	valid_0's multi_error: 0.554413
[636]	valid_0's multi_error: 0.55441
[637]	valid_0's multi_error: 0.554374
[638]	valid_0's multi_error: 0.554353
[639]	valid_0's multi_error: 0.554296
[640]	valid_0's multi_error: 0.554281
[641]	valid_0's multi_error: 0.554241
[642]	valid_0's

[833]	valid_0's multi_error: 0.553543
[834]	valid_0's multi_error: 0.553525
[835]	valid_0's multi_error: 0.553513
[836]	valid_0's multi_error: 0.553534
[837]	valid_0's multi_error: 0.553541
[838]	valid_0's multi_error: 0.553528
[839]	valid_0's multi_error: 0.553489
[840]	valid_0's multi_error: 0.553472
[841]	valid_0's multi_error: 0.553538
[842]	valid_0's multi_error: 0.553528
[843]	valid_0's multi_error: 0.553525
[844]	valid_0's multi_error: 0.553452
[845]	valid_0's multi_error: 0.553447
[846]	valid_0's multi_error: 0.553428
[847]	valid_0's multi_error: 0.553432
[848]	valid_0's multi_error: 0.553422
[849]	valid_0's multi_error: 0.553401
[850]	valid_0's multi_error: 0.553374
[851]	valid_0's multi_error: 0.553374
[852]	valid_0's multi_error: 0.553389
[853]	valid_0's multi_error: 0.553402
[854]	valid_0's multi_error: 0.553392
[855]	valid_0's multi_error: 0.553405
[856]	valid_0's multi_error: 0.553401
[857]	valid_0's multi_error: 0.553377
[858]	valid_0's multi_error: 0.553411
[859]	valid_

[1048]	valid_0's multi_error: 0.552929
[1049]	valid_0's multi_error: 0.552902
[1050]	valid_0's multi_error: 0.552923
[1051]	valid_0's multi_error: 0.552896
[1052]	valid_0's multi_error: 0.552908
[1053]	valid_0's multi_error: 0.552881
[1054]	valid_0's multi_error: 0.552871
[1055]	valid_0's multi_error: 0.552853
[1056]	valid_0's multi_error: 0.552816
[1057]	valid_0's multi_error: 0.552799
[1058]	valid_0's multi_error: 0.552793
[1059]	valid_0's multi_error: 0.552753
[1060]	valid_0's multi_error: 0.552752
[1061]	valid_0's multi_error: 0.552752
[1062]	valid_0's multi_error: 0.552749
[1063]	valid_0's multi_error: 0.552737
[1064]	valid_0's multi_error: 0.552764
[1065]	valid_0's multi_error: 0.552775
[1066]	valid_0's multi_error: 0.552783
[1067]	valid_0's multi_error: 0.552783
[1068]	valid_0's multi_error: 0.552784
[1069]	valid_0's multi_error: 0.552766
[1070]	valid_0's multi_error: 0.55274
[1071]	valid_0's multi_error: 0.552734
[1072]	valid_0's multi_error: 0.55275
[1073]	valid_0's multi_erro

[1259]	valid_0's multi_error: 0.552392
[1260]	valid_0's multi_error: 0.55242
[1261]	valid_0's multi_error: 0.552405
[1262]	valid_0's multi_error: 0.552411
[1263]	valid_0's multi_error: 0.552441
[1264]	valid_0's multi_error: 0.552472
[1265]	valid_0's multi_error: 0.552469
[1266]	valid_0's multi_error: 0.552469
[1267]	valid_0's multi_error: 0.552464
[1268]	valid_0's multi_error: 0.552434
[1269]	valid_0's multi_error: 0.55244
[1270]	valid_0's multi_error: 0.552425
[1271]	valid_0's multi_error: 0.552435
[1272]	valid_0's multi_error: 0.552447
[1273]	valid_0's multi_error: 0.552426
[1274]	valid_0's multi_error: 0.552425
[1275]	valid_0's multi_error: 0.552401
[1276]	valid_0's multi_error: 0.55241
[1277]	valid_0's multi_error: 0.552408
[1278]	valid_0's multi_error: 0.552381
[1279]	valid_0's multi_error: 0.552381
[1280]	valid_0's multi_error: 0.552384
[1281]	valid_0's multi_error: 0.552384
[1282]	valid_0's multi_error: 0.552419
[1283]	valid_0's multi_error: 0.552426
[1284]	valid_0's multi_error

C:\Users\13224\Anaconda2\lib\site-packages\ipykernel_launcher.py:44: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [20]:
age_train['age_group'].nunique()

6

In [21]:
history

print("训练模型：")
param = {
        'learning_rate': 0.1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.9,
        'metric': 'multi_error'
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=1500, valid_sets=[vali], early_stopping